In [ ]:
import requests
import re
import string
from textblob import TextBlob
import nltk
# nltk.download('all')
from nltk.corpus import stopwords
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
import shutil
import os, sys
import numpy as np
import pandas as pd
import zipfile
# !pip install contractions
import contractions
from bs4 import BeautifulSoup
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM,Dropout
from tensorflow.keras.optimizers import SGD
import sqlite3
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Source path: kaggle.json file in Google Drive
source_path = '/content/drive/MyDrive/kaggle token/kaggle.json'

# Destination path: Colab environment
destination_path = '/content/kaggle.json'

# Copy the file from Google Drive to Colab
shutil.copyfile(source_path, destination_path)

# List files in the current directory
print(os.listdir('/content'))

['.config', 'drive', 'database.sqlite', 'X_array.npy', 'kaggle.json', 'sample_data']


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

100% 242M/242M [00:08<00:00, 32.8MB/s]
100% 242M/242M [00:08<00:00, 29.4MB/s]


In [ ]:
with zipfile.ZipFile('/content/amazon-fine-food-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
connection = sqlite3.connect("/content/database.sqlite")

In [ ]:
filtered_data = pd.read_sql_query("""select * from reviews""", connection)
filtered_data.shape

(568454, 10)

In [ ]:
df=filtered_data.copy()
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [ ]:
df.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'Time', 'Summary'], axis=1, inplace=True)

In [ ]:
df.drop(df[df.HelpfulnessNumerator > df.HelpfulnessDenominator].index,axis=0,inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df.drop(['HelpfulnessNumerator', 'HelpfulnessDenominator'],inplace=True,axis=1)

In [ ]:
df=df.reindex(columns=['Text', 'Score'])

In [ ]:
df

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5
...,...,...
568447,Great for sesame chicken..this is a good if no...,5
568448,I'm disappointed with the flavor. The chocolat...,2
568449,"These stars are small, so you can give 10-15 o...",5
568450,These are the BEST treats for training and rew...,5


In [ ]:
df.drop(df[df.duplicated()].index,axis=0,inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)
df

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5
...,...,...
393668,Great for sesame chicken..this is a good if no...,5
393669,I'm disappointed with the flavor. The chocolat...,2
393670,"These stars are small, so you can give 10-15 o...",5
393671,These are the BEST treats for training and rew...,5


In [ ]:
df.drop(df[df['Score'] == 3].index,axis=0,inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)
df

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5
...,...,...
363896,Great for sesame chicken..this is a good if no...,5
363897,I'm disappointed with the flavor. The chocolat...,2
363898,"These stars are small, so you can give 10-15 o...",5
363899,These are the BEST treats for training and rew...,5


In [ ]:
df.Score.value_counts()

5    250744
4     56073
1     36280
2     20804
Name: Score, dtype: int64

In [ ]:
def mylabel(x):
    if x<3:
        return 0
    return 1

In [ ]:
ActaulScore = df.Score
df.Score = df.Score.apply(lambda x: mylabel(x))
df['ActaulScore'] = ActaulScore

In [ ]:
df.head()

,Text,Score,ActaulScore
0,I have bought several of the Vitality canned d...,1,5
1,Product arrived labeled as Jumbo Salted Peanut...,0,1
2,This is a confection that has been around a fe...,1,4
3,If you are looking for the secret ingredient i...,0,2
4,Great taffy at a great price. There was a wid...,1,5


In [ ]:
df.Score.value_counts()

1    306817
0     57084
Name: Score, dtype: int64

In [ ]:
# 1. Lowercasing:
def lowercasing(text):

    return text.lower()

# 2. Remove HTML TAGS:
def remove_htlm_tags(text):
    p_text = BeautifulSoup(text, 'html.parser',).get_text()
    pattern_html = re.compile('<.*?>')
    return pattern_html.sub(r'', p_text)

# 3. Remove URLs
def remove_urls(text):

    pattern_url = re.compile(r'https?://\S+|www\.\S+')
    return pattern_url.sub(r'', text)

# 4. Remove Punctuation:
def remove_punctuation(text):

    exclude = string.punctuation
    return text.translate(str.maketrans('', '', exclude))

# 5. Chat Word Treatment:
chat_words = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

def remove_chat_words(text):

    new_text = []
    for w in text.split():
        if w.lower() in chat_words:
            new_text.append(chat_words[w.lower()])
        else:
            new_text.append(w)
    return " ".join(new_text)

# 6 Handling Emojis
def remove_emojis(text):

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# 7. Removing Stopwords
def remove_stopwords(text):

    new_text = []

    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return  " ".join(x)

# 8. Spelling Correction:
def spell_check(text):

    my_list = []

    for word in text.split():
        textBlb = TextBlob(word)
        my_list.append(textBlb.correct().string)

    return " ".join(my_list)

# 9. Lemmatization
def Lemmatize_text(text):

    wordnet_lemmatizer = WordNetLemmatizer()

    lemm_text = " ".join([wordnet_lemmatizer.lemmatize(word) for word in text.split()])
    return lemm_text

# 10 Tokenization
def tokenize_text(text):

    return word_tokenize(text)

# 11. Contractions
def decontract_word(text):
    return contractions.fix(text)

In [ ]:
df.Text[57046]

'*********************************************************<br /> UPDATE:  READ THE UPDATE BELOW FIRST. Thanks. (June 15 and August 3, 2012)<br />*********************************************************<br />  I WAS, BUT NO MORE (actively anyway). We have four cats: two ("normals") who eat about anything, one with kidney disease (CRF/CKD) and accompanying reduced appetite, and one that has picky (and weird) tastes.  In the process of trying to find an appetite stimulant for the mom-cat (with CRF/CKD) I tried out several products on her and the others (the kids) as well.<br />Here are the results of this less than scientific (or definitive) survey:<br /><br />  Drs.Foster&Smith Shrimp Snappies Treats (ground to powder)-----------ALL REFUSED.<br />  Freeze-dried powders (Prowl and Ziwi Peak)----------------------------NO EFFECT. All but one (Ms. Picky) ate them.<br />  Seafood juices (tuna, oyster, sardine, etc.)-----------------------------NO EFFECT. All but (same) one ate them.<br />  

In [ ]:
ActualText = df.Text

In [ ]:
df.Text = df.Text.apply(lambda x: lowercasing(x))
df.Text = df.Text.apply(lambda x: remove_htlm_tags(x))
df.Text = df.Text.apply(lambda x: remove_urls(x))
df.Text = df.Text.apply(lambda x: remove_punctuation(x))
df.Text = df.Text.apply(lambda x: decontract_word(x))
df.Text = df.Text.apply(lambda x: remove_chat_words(x))
df.Text = df.Text.apply(lambda x: remove_emojis(x))

<ipython-input-56-29db8ea06ecb>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  p_text = BeautifulSoup(text, 'html.parser',).get_text()


In [ ]:
df.Text = df.Text.apply(lambda x: remove_stopwords(x))

In [ ]:
df.to_csv('data_after_stopwords.csv', index=False)

In [ ]:
df.Text[57046]

'update read  update  first thanks june 15  august 3 2012      actively anyway   four cats two normals  eat  anything one  kidney disease crfckd  accompanying reduced appetite  one   picky  weird tastes   process  trying  find  appetite stimulant   momcat  crfckd  tried  several products     others  kids  wellhere   results   less  scientific  definitive survey drsfostersmith shrimp snappies treats ground  powderall refused freezedried powders prowl  ziwi peakno effect   one miss picky ate  seafood juices tuna oyster sardine etcno effect    one ate  dry ground  powder cat food halo evo blue fuffalo etcno effect ditto acceptance canned cat food gourmet brands renal dietsditto hairball tube lubes usual suspects cat grassditto markchappells de furrums treatsditto calming chews liquid pet naturalsditto except miss picky accepted brewers yeastpowderdulled  appetites  bit  accepted barley cat greenfoods powdereffective  momcat others liked exceptdrsfostersmith tuna treats ground  powdereffec

In [ ]:
# Source path: kaggle.json file in Google Drive
source_path = '/content/drive/MyDrive/kaggle token/Amazon Fine Food Reviews/data_after_stopwords.csv'

# Destination path: Colab environment
destination_path = '/content/data_after_stopwords.csv'

# Copy the file from Google Drive to Colab
shutil.copyfile(source_path, destination_path)

# List files in the current directory
print(os.listdir('/content'))

['.config', 'hashes.txt', 'drive', 'Reviews.csv', 'database.sqlite', 'X_array.npy', 'data_after_stopwords.csv', 'amazon-fine-food-reviews.zip', 'sample_data']


In [ ]:
df=pd.read_csv('/content/data_after_stopwords.csv')

In [ ]:
df.Text[57046]

'update read  update  first thanks june 15  august 3 2012      actively anyway   four cats two normals  eat  anything one  kidney disease crfckd  accompanying reduced appetite  one   picky  weird tastes   process  trying  find  appetite stimulant   momcat  crfckd  tried  several products     others  kids  wellhere   results   less  scientific  definitive survey drsfostersmith shrimp snappies treats ground  powderall refused freezedried powders prowl  ziwi peakno effect   one miss picky ate  seafood juices tuna oyster sardine etcno effect    one ate  dry ground  powder cat food halo evo blue fuffalo etcno effect ditto acceptance canned cat food gourmet brands renal dietsditto hairball tube lubes usual suspects cat grassditto markchappells de furrums treatsditto calming chews liquid pet naturalsditto except miss picky accepted brewers yeastpowderdulled  appetites  bit  accepted barley cat greenfoods powdereffective  momcat others liked exceptdrsfostersmith tuna treats ground  powdereffec

In [ ]:
df.Text = df.Text.apply(lambda x: Lemmatize_text(x))

In [ ]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
story = []
for doc in df['Text']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [ ]:
len(story)

363901

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(63838270, 69442495)

In [ ]:
len(model.wv.index_to_key)

89522

In [ ]:
import pickle

with open("embedding.pkl", 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Source path: kaggle.json file in Google Drive
destination_path = '/content/drive/MyDrive/kaggle token/Amazon Fine Food Reviews/embedding.pkl'

# Destination path: Colab environment
source_path = '/content/embedding.pkl'

# Copy the file from Google Drive to Colab
shutil.copyfile(source_path, destination_path)

'/content/drive/MyDrive/kaggle token/Amazon Fine Food Reviews/embedding.pkl'

In [ ]:
model.wv.index_to_key

['like',
 'taste',
 'good',
 'product',
 'one',
 'flavor',
 'great',
 'love',
 'tea',
 'coffee',
 'would',
 'food',
 'get',
 'make',
 'time',
 'really',
 'use',
 'much',
 'amazon',
 'also',
 'little',
 'price',
 'best',
 'find',
 'dog',
 'buy',
 'bag',
 'tried',
 'even',
 'well',
 'store',
 'try',
 'better',
 'chocolate',
 'box',
 'day',
 'eat',
 'year',
 'drink',
 'sugar',
 'water',
 'first',
 'used',
 'could',
 'cup',
 'found',
 'brand',
 'go',
 'way',
 'sweet',
 'made',
 'bought',
 'mix',
 'cannot',
 'treat',
 'give',
 'think',
 'order',
 'thing',
 'two',
 'add',
 'bit',
 'since',
 'delicious',
 'say',
 'know',
 'cat',
 'recommend',
 'lot',
 'favorite',
 'want',
 'many',
 'still',
 'nice',
 'got',
 'work',
 'never',
 'hot',
 'pack',
 'free',
 'milk',
 'bar',
 'need',
 'stuff',
 'ingredient',
 'keep',
 'come',
 'snack',
 'every',
 'always',
 'without',
 'sauce',
 'something',
 'oil',
 'ever',
 'right',
 'organic',
 'quality',
 'review',
 'take',
 'perfect',
 'le',
 'back',
 'easy',
 

In [ ]:
embedding_dim = model.vector_size
embedding_dim

100

In [ ]:
vocab_size = len(model.wv.index_to_key)
vocab_size

89522

In [ ]:
word2vec_embeddings = model.wv.vectors
word2vec_embeddings

array([[ 9.2774534e-01, -1.5023787e+00,  7.4053675e-01, ...,
        -2.1116335e+00, -4.3844080e-01, -6.7172092e-01],
       [-3.1783082e+00,  2.8324929e-01, -1.1783381e+00, ...,
         7.3244441e-01, -3.2999737e+00, -6.4130461e-01],
       [ 1.3003731e+00, -1.3869165e+00,  3.9605382e-01, ...,
        -2.3403137e+00,  1.3378174e+00, -2.7454779e+00],
       ...,
       [ 6.0134742e-02,  3.5988390e-02, -6.5530486e-02, ...,
        -4.1740734e-02, -6.1622195e-02,  1.1339101e-02],
       [-1.4925815e-02,  3.7967306e-02, -5.2599739e-02, ...,
        -5.6123231e-02, -2.1821633e-03,  4.9175196e-03],
       [-4.7635596e-02,  4.6493165e-02, -3.7969656e-02, ...,
         1.9539181e-02,  1.8433726e-03,  1.1518774e-02]], dtype=float32)

In [ ]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    # return np.mean(model.wv[doc], axis=0)
    if len(doc) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(model.wv[doc], axis=0)

In [ ]:
from tqdm import tqdm

In [ ]:
X = []
for doc in tqdm(df['Text'].values):
    X.append(document_vector(doc))

100%|██████████| 363901/363901 [1:50:22<00:00, 54.95it/s] 


In [ ]:
# Source path: kaggle.json file in Google Drive
source_path = '/content/drive/MyDrive/kaggle token/Amazon Fine Food Reviews/X_array.npy'

# Destination path: Colab environment
destination_path = '/content/X_array.npy'

# Copy the file from Google Drive to Colab
shutil.copyfile(source_path, destination_path)

# List files in the current directory
print(os.listdir('/content'))

['.config', 'drive', 'X_array.npy', 'sample_data']


In [ ]:
# Load array from file
X = np.load('/content/X_array.npy')

In [ ]:
X_array = np.array(X)

In [ ]:
# np.save("X_array.npy", X_array)

In [ ]:
# # Destination path: Colab environment
# destination_path = '/content/drive/MyDrive/kaggle token/Amazon Fine Food Reviews/X_array.npy'

# # Source path: kaggle.json file in Google Drive
# source_path = '/content/X_array.npy'

# # Copy the file from Google Drive to Colab
# shutil.copyfile(source_path, destination_path)

'/content/drive/MyDrive/kaggle token/Amazon Fine Food Reviews/X_array.npy'

In [ ]:
len(X_array)

363901

In [ ]:
y = df.Score

In [ ]:
len(y)

363901

In [ ]:
y.value_counts(normalize=True)

1    0.843133
0    0.156867
Name: Score, dtype: float64

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

In [ ]:
X_resampled, y_resampled = smote.fit_resample(X_array, y)

In [ ]:
X_resampled.shape

(613634, 100)

In [ ]:
y_resampled.value_counts()

1    306817
0    306817
Name: Score, dtype: int64

In [ ]:
y_resampled.value_counts(normalize=True)

1    0.5
0    0.5
Name: Score, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_resampled, y_resampled, test_size=0.2,random_state=100)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(490907, 100) (122727, 100) (490907,) (122727,)


In [ ]:
y_train.value_counts()

1    245647
0    245260
Name: Score, dtype: int64

In [ ]:
y_test.value_counts()

0    61557
1    61170
Name: Score, dtype: int64

In [ ]:
X_train.shape

(490907, 100)

In [ ]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.361666,-0.649384,-0.165163,0.464911,0.240999,0.408452,0.161752,-0.308742,0.503308,0.021140,...,-0.503528,0.945742,1.357465,0.384178,-0.158305,-0.281029,-0.768853,-0.247017,0.000988,-0.376036
1,-0.199758,0.386887,-0.532587,0.094897,0.550664,0.735532,-0.372645,0.099243,-0.564690,1.134064,...,-0.997956,-0.385426,0.508384,0.193647,-0.581295,-0.532201,1.271928,-1.116990,0.846998,0.183547
2,-0.537206,-1.192630,-0.602384,0.478485,0.065365,-0.040328,0.155729,0.491419,0.453497,0.741548,...,-0.128561,0.265893,0.932871,0.667412,0.397642,-0.175287,-0.215412,0.516531,-0.655636,-0.936930
3,0.245311,-0.292232,0.269039,-0.177600,-0.183054,0.313011,-0.140608,-0.239761,0.531568,0.571707,...,-0.892973,-0.029645,0.948892,-0.078993,-0.330061,-0.367557,0.372012,-0.942887,-0.093441,0.155026
4,-0.766812,-0.441639,0.542326,-0.152748,0.559029,0.373853,0.363605,0.003740,0.447539,0.452493,...,-0.651264,0.142484,1.420395,-0.129240,0.422716,-0.189979,-0.208132,-0.207771,-0.184688,0.343495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490902,0.877517,0.139944,0.199321,-0.911610,0.062562,0.998103,-0.297127,0.801820,0.994604,1.498863,...,-0.870632,-0.632923,1.422792,0.603066,-0.906010,-0.900007,0.361793,0.264670,0.005507,-0.694226
490903,0.056596,-0.110151,-0.100029,1.034876,0.024800,-0.088492,-0.027180,0.098046,0.143772,0.491855,...,0.185231,-1.195459,0.128154,0.148818,0.008531,-0.618350,-0.089460,-0.055081,-0.302159,-0.249157
490904,0.161622,-0.575298,-0.709764,0.412323,-1.235389,0.277500,-0.987903,-0.886399,0.893830,0.616606,...,-0.182409,0.253699,0.050778,0.032131,-0.267368,-1.143931,0.361381,0.180575,-0.948684,0.583458
490905,-0.025858,0.366417,-1.033963,0.983283,-1.772663,-0.315896,0.806780,0.099389,0.108990,0.431698,...,-0.309070,1.006871,-0.084772,-0.551999,-0.721717,0.452178,-0.099828,-1.303208,0.299936,-0.302383


#**Deep learning Model**

In [ ]:
# Define DNN model
DNN = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),  # Optional dropout layer for regularization
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile model
DNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

# Train model
DNN.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test,y_test), callbacks=[early_stopping])

# Evaluate model
loss, accuracy = DNN.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/100
15341/15341 [==============================] - 68s 4ms/step - loss: 0.3034 - accuracy: 0.8760 - val_loss: 0.2668 - val_accuracy: 0.8901
Epoch 2/100
15341/15341 [==============================] - 63s 4ms/step - loss: 0.2829 - accuracy: 0.8855 - val_loss: 0.2593 - val_accuracy: 0.8942
Epoch 3/100
15341/15341 [==============================] - 74s 5ms/step - loss: 0.2774 - accuracy: 0.8877 - val_loss: 0.2571 - val_accuracy: 0.8941
Epoch 4/100
15341/15341 [==============================] - 69s 5ms/step - loss: 0.2731 - accuracy: 0.8897 - val_loss: 0.2579 - val_accuracy: 0.8955
Epoch 5/100
15341/15341 [==============================] - 62s 4ms/step - loss: 0.2707 - accuracy: 0.8913 - val_loss: 0.2516 - val_accuracy: 0.8982
Epoch 6/100
15341/15341 [==============================] - 68s 4ms/step - loss: 0.2690 - accuracy: 0.8923 - val_loss: 0.2548 - val_accuracy: 0.8968
Epoch 7/100
15341/15341 [==============================] - 66s 4ms/step - loss: 0.2672 - accuracy: 0.8930 - val_

In [ ]:
with open("DNN.pkl", 'wb') as f:
    pickle.dump(DNN, f)

In [ ]:
# Destination path: Colab environment
destination_path = '/content/drive/MyDrive/kaggle token/Amazon Fine Food Reviews/DNN.pkl'

# Source path: kaggle.json file in Google Drive
source_path = '/content/DNN.pkl'

# Copy the file from Google Drive to Colab
shutil.copyfile(source_path, destination_path)

'/content/drive/MyDrive/kaggle token/Amazon Fine Food Reviews/DNN.pkl'

In [ ]:
mytxt = """
Its seriously a waste of money. I have tried all most all variants of Yummiez Chicken sausages, but found their combinations are very bad. Taste, Smell and even its quality of chicken is not good. " A Sausage manufacturer who don't know what is sausage". Picture on the packet is yummy, But not the product inside.
Don't buy and get disappoint with this product.
"""

In [ ]:
mytxt = lowercasing(mytxt)
mytxt = remove_chat_words(mytxt)
mytxt = remove_htlm_tags(mytxt)
mytxt = remove_urls(mytxt)
mytxt = remove_punctuation(mytxt)
mytxt = decontract_word(mytxt)
mytxt = remove_emojis(mytxt)
mytxt = remove_stopwords(mytxt)
mytxt = Lemmatize_text(mytxt)

In [ ]:
mytxt

'seriously waste money tried variant yummiez chicken sausage found combination bad taste smell even quality chicken good sausage manufacturer know sausage picture packet yummy product inside buy get disappoint product'

In [ ]:
mytxt1 = document_vector(mytxt)

In [ ]:
mytxt1 = mytxt1.reshape(1, -1)

In [ ]:
mytxt1.shape

(1, 100)

In [ ]:
predictions = DNN.predict(mytxt1)

1/1 [==============================] - 0s 43ms/step


In [ ]:
binary_predictions = (predictions > 0.5).astype(int)
if binary_predictions ==1:
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Negative Sentiment
